In [22]:
import json
import os
import numpy as np
import pandas as pd

root = "verified"

identifier = []
photos = []
captions = []
collections = 0
no_image = 0
no_caption = 0
has_quotes = 0
scraper_failed = 0
posts_per_user = []
for fn in os.listdir("profiles_" + root + "/"):
    if fn.endswith(".json"):
        f = open("profiles_" + root + "/" + fn)
        num_valid_entries = 0
        num_invalid_entries = 0
        data = json.load(f)
        try:
            posts = data["posts"]
        except:
            if data["num_of_posts"] > 0:
                scraper_failed += data["num_of_posts"]
            exit
        for p in posts:
            if len(p["imgs"]) > 1:
                collections += 1
                num_invalid_entries += 1
            elif len(p["imgs"]) == 0:
                no_image += 1
                num_valid_entries += 1
            elif p["caption"] == "":
                no_caption += 1
                num_invalid_entries += 1
            else:
                num_valid_entries += 1
                identifier.append(data["username"])
                photos.append(p["imgs"][0])
                if '"' in p["caption"]:
                    has_quotes += 1
                captions.append(p["caption"].replace('"', '\''))
        posts_per_user.append(num_valid_entries)
        scraper_failed += data["num_of_posts"] - num_valid_entries - num_invalid_entries
        f.close()

total = len(captions) + collections + no_image + no_caption
print("scraper failed", scraper_failed, round(scraper_failed * 100/ (total + scraper_failed), 2))
print("throwing out:", collections + no_image + no_caption, round((collections + no_image + no_caption) * 100 / total, 2))
print("\t no caption", no_caption, round((no_caption / total) * 100, 2))
print("\t no image", no_image, round((no_image / total) * 100, 2))
print("\t collection", collections, round((collections / total) * 100, 2))
print("usable:", len(captions), round(len(captions)*100/total, 2))
print("post per user:", posts_per_user)

with open('data_' + root + '.csv', 'w') as f:
    f.write("username,photo,caption\n")
    for i in range(len(photos)):
        f.write("%s,%s,\"%s\"\n" % (identifier[i],photos[i], captions[i]))
f.close()

try:
    csv = pd.read_csv('data_' + root + '.csv', header=0)
    csv.drop_duplicates(keep="last",inplace=True)
    csv.to_csv('data_' + root + '.csv', index=False)
except:
    print('pandas stuff did not work')


scraper failed 55702 95.52
throwing out: 1015 38.89
	 no caption 94 3.6
	 no image 587 22.49
	 collection 334 12.8
usable: 1595 61.11
post per user: [69, 93, 88, 0, 70, 83, 88, 36, 60, 100, 7, 93, 70, 7, 86, 80, 82, 96, 85, 75, 83, 70, 0, 88, 80, 11, 11, 72, 11, 20, 82, 82, 100, 94, 10]
